<a href="https://colab.research.google.com/github/thaiphi/Stock-Markets-Analytics-2024/blob/main/Week%202/Module_2_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


##Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happenned of Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/


1. Convert the 'Filing Date' to datetime()
2. 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)
3. Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given)You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string.
4. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
5. Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [3]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [4]:
ipo_fillings = ipo_dfs[0]

ipo_fillings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     326 non-null    object
 1   Symbol          326 non-null    object
 2   Company Name    326 non-null    object
 3   Price Range     326 non-null    object
 4   Shares Offered  326 non-null    object
dtypes: object(5)
memory usage: 12.9+ KB


In [5]:
ipo_fillings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
1,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
2,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
3,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,"Apr 16, 2024",JLJT,Jialiang Holdings Ltd,$5.00,-


##Cleaning the data

In [6]:
# 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs)

import numpy as np
ipo_fillings['Shares Offered'] = ipo_fillings['Shares Offered'].apply(lambda x: float(x) if x != '-' else np.nan)


In [7]:
# Convert 'Filing Date' to datetime
ipo_fillings['Filing Date'] = pd.to_datetime(ipo_fillings['Filing Date'])


In [24]:
# Problem --> not always the columns are filled
missing_prices_df = ipo_fillings[ipo_fillings['Price Range'].astype(str).str.find('-') >= 0]
missing_prices_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
5,2024-04-16,GGL,Games Global Limited,-,NaN
6,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN
7,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN
8,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0
...,...,...,...,...,...
319,2020-06-17,ALEH,ALE Group Holding Limited,$4.00 - $6.00,4200000.0
320,2020-02-18,CAST,"FreeCast, Inc.",-,NaN
321,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
323,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0


In [25]:
#Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified,
#to the price (if only one number is provided), or to the average of 2 prices (if a range is given)
def str_to_digits(s):
    if not s or s == '-':  # If empty string or just a dash
        return np.nan
    s = s.replace('$', '').replace(',', '')  # Remove $ and commas
    if '-' in s:  # If it's a range
        try:
            start, end = map(float, s.split('-'))
            return (start + end) / 2  # Return average of the range
        except ValueError:
            return np.nan  # Return nan if conversion fails
    else:
        try:
            return float(s)  # Convert to float
        except ValueError:
            return np.nan  # Return nan if conversion fails

# Apply the function to the 'Price Range' column
ipo_fillings['Avg_price'] = ipo_fillings['Price Range'].apply(lambda x: str_to_digits(x))

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
1    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
2    2024-04-22   DRJT                        Derun Group Inc           $5.00   
3    2024-04-19   GPAT           GP-Act III Acquisition Corp.          $10.00   
4    2024-04-16   JLJT                  Jialiang Holdings Ltd           $5.00   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [26]:
# Define a column "Shares_offered_value",
# which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)
def calculate_shares_offered_value(row):
    shares_offered = row['Shares Offered']
    avg_price = row['Avg_price']
    if pd.notnull(shares_offered) and pd.notnull(avg_price):
        return shares_offered * avg_price
    else:
        return np.nan

# Apply the function to create the Shares_offered_value column
ipo_fillings['Shares_offered_value'] = ipo_fillings.apply(calculate_shares_offered_value, axis=1)

print(ipo_fillings)

    Filing Date Symbol                           Company Name     Price Range  \
0    2024-04-26   EURK                Eureka Acquisition Corp          $10.00   
1    2024-04-26    HDL    Super Hi International Holding Ltd.               -   
2    2024-04-22   DRJT                        Derun Group Inc           $5.00   
3    2024-04-19   GPAT           GP-Act III Acquisition Corp.          $10.00   
4    2024-04-16   JLJT                  Jialiang Holdings Ltd           $5.00   
..          ...    ...                                    ...             ...   
321  2020-01-21   GOXS                            Goxus, Inc.  $8.00 - $10.00   
322  2020-01-21   UTXO                 UTXO Acquisition, Inc.          $10.00   
323  2019-12-09   LOHA                           Loha Co. Ltd  $8.00 - $10.00   
324  2019-10-04   ZGHB  China Eco-Materials Group Co. Limited           $4.00   
325  2018-12-27   FBOX              Fit Boxx Holdings Limited   $5.00 - $6.50   

     Shares Offered  Avg_pr

In [28]:
# Find the total sum in $m (millions of USD, closest INTEGER number) for all fillings during 2023,
# which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 24 of it is not null.
friday_fillings = ipo_fillings[(ipo_fillings['Filing Date'].dt.dayofweek == 4) & (ipo_fillings['Filing Date'].dt.year == 2023)]
total_sum_in_millions = int(friday_fillings['Shares_offered_value'].sum() / 1000000)
print(f"Total sum of 2023 filings that happened on Fridays: ${total_sum_in_millions}m")


Total sum of 2023 filings that happened on Fridays: $285m


In [ ]:
# it has some missing values --> use defensive errors='coerce' (if don't have time to crack into the data errors)
#     : pd.to_numeric() function call, which will convert problematic values to NaN.
#     otherwise you'll get a ValueError: Unable to parse string "-" at position 9
ipos_2023['IPO Price'] = pd.to_numeric(ipos_2023['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
ipos_2023['IPO Price'] = pd.to_numeric(ipos_2023['IPO Price'])

In [ ]:
# Convert "Current" column
ipos_2023['Current'] = pd.to_numeric(ipos_2023['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
ipos_2023['Return'] = pd.to_numeric(ipos_2023['Return'].str.replace('%', ''), errors='coerce') / 100

In [ ]:
# simple way of checking NULLs
# (you need to understand how vector operations work .isnull() and calls chaining .isnull().sum())
ipos_2023.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       0
Current         0
Return          0
dtype: int64